<a href="https://colab.research.google.com/github/heejininku/class2022Fall/blob/main/20221111(2)_YES24API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

교보문고보다 복잡함.


여기는 예쁘게 정리 안되어있음

In [ ]:
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup #크롤링할 때 체계적이지 않을 때에 사용하는 BeautifulSoup #soup:잡탕 - 아름답게 정리하는 것
import re
import numpy as np


In [ ]:
bookID = '176787' #사이트 위에 북 아이디 있음.

In [ ]:
url = f'http://www.yes24.com/Product/communityModules/GoodsReviewList/{bookID}?Sort=1&PageNumber=1&Type=ALL'
#여기도 요청임. 
response = requests.get(url) 
#그냥 response하면 (response.text하면 내부 보여주는데 너무 지저분) 지저분해서
#아래처럼 soup로 어느정도 정리
soup = BeautifulSoup(response.content, 'html.parser')
tmp = soup.find(string=re.compile('reviewCountText'))
tmp = str(tmp)
cnt = re.findall('(?<= reviewCountText = \')\d*(?=\')', tmp)
cnt = int(cnt[0])

nPage = int(np.ceil(cnt/5))

In [ ]:
data = {'title': [],
        'ID': [],
        'date': [],
        'contRating': [],
        'designRating': [],
        'text': []}
df = pd.DataFrame(data)

아래 복잡해보여도 구조로는 모르는 게 없어야 함.

In [ ]:
i = 0
for n in range(nPage):
  PageNumber = n + 1
  url = f'http://www.yes24.com/Product/communityModules/GoodsReviewList/{bookID}?Sort=1&PageNumber={PageNumber}&Type=ALL'
  response = requests.get(url) #입력이 url
  
  soup = BeautifulSoup(response.content, 'html.parser')
  title = soup.select('span.txt')
  ID = soup.select('a.lnk_id')
  date = soup.select('em.txt_date')
  rating = soup.select('span.review_rating > span.rating');
  contRating = rating[0:9:2]; designRating = rating[1:10:2]
  text = soup.select('div.reviewInfoBot.origin > div.review_cont')

  for r in zip(title, ID, date, contRating, designRating, text):
    title, ID, date, contRating, designRating, text = [r[0].get_text(), r[1].get_text(), r[2].get_text(), r[3].get_text(), r[4].get_text(), r[5].get_text()]    
    row = [title, ID, date, contRating, designRating, text]
    df.loc[i, :] = row
    i +=1

In [ ]:
import xlwt
df.to_excel('review.xls')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  


엑셀 안에 제목, 아이디, 날짜, 리뷰 등

나중에 텍스트 가지고 분석해보기
nlp 자연어처리 활용

구글에 자연어처리 검색도 하심.
기본적으로 텍스트 처리하는 것.
nltk 자연어처리_테이터 사이언스 스쿨
교수님 유튜브 강의도 있음.